<a href="https://colab.research.google.com/github/yannick5000/Fake-News-Detector-Covid-19-vaccine/blob/main/Project_FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Health-related fakenews detector – Project Work**



In [7]:
# import modules
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import string

# loading the dataset and removing punctuation
df = pd.read_csv("statements3.csv", index_col="Index")
df["Statement"]=df["Statement"].str.replace("[^\w\s]","")
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


,Statement,Source,Source (URL),Date + time,True/False,Evidence (why true?),Category
Index,,,,,,,
659,Der Impfstoff kann eine Covid19Erkrankung vers...,BR24,https://www.br.de/nachrichten/wissen/fake-news...,2021-07-19 16-21,False,Lack of understanding complex medical/biologic...,Consequences of vaccination
438,Vorläufige Erkenntnisse deuten darauf hin dass...,RKI,https://www.rki.de/SharedDocs/FAQ/COVID-Impfen...,2022-05-24,True,NaN,NaN
842,Bewohner eines Pfelgeheims der Diakonie in Sac...,Facts For Friends,https://www.factsforfriends.de/fact/5ff85b479a...,2021-01-08,False,Falsche Behauptung,Vaccination mandate
498,Kinder mit Vorerkrankungen sollen weiterhin ei...,RKI,https://www.rki.de/SharedDocs/FAQ/COVID-Impfen...,2022-06-11,True,NaN,NaN
490,Darüber hinaus sind Abstand halten Maske trage...,RKI,https://www.rki.de/SharedDocs/FAQ/COVID-Impfen...,2022-06-03,True,NaN,NaN


## Binary Coding


In [10]:
 
a=df["True/False"].eq(1).groupby(df["Statement"]).mean().reset_index()
a.head()

,Statement,True/False
0,An COVID würden fast nur diejenigen Menschen ...,0.0
1,Azithromycin kann das CovidVirus abtöten,0.0
2,Bleichmittel zu trinken um das Virus im Körpe...,0.0
3,Chloroquin kann das CovidVirus abtöten,0.0
4,Das Virus wurde künstlich hergestellt und die...,0.0


## Removing whitespace and writing everything in lower case 

In [ ]:

df["Statement"] = df["Statement"].apply(lambda x: x.lower() ) 
df["Statement"]= df["Statement"].apply(lambda x: x.strip()) 

In [11]:
# Tokenization : bring sentences into world
import nltk
nltk.download("punkt")
nltk.download("wordnet")

df["Statement"]= df["Statement"].apply(lambda x: nltk.word_tokenize(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [12]:
# import the stop word
            # german stop word fron nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

stopWordsGerman = stopwords.words("german")
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Statement,Source,Source (URL),Date + time,True/False,Evidence (why true?),Category
Index,,,,,,,
659,"[Der, Impfstoff, kann, eine, Covid19Erkrankung...",BR24,https://www.br.de/nachrichten/wissen/fake-news...,2021-07-19 16-21,False,Lack of understanding complex medical/biologic...,Consequences of vaccination
438,"[Vorläufige, Erkenntnisse, deuten, darauf, hin...",RKI,https://www.rki.de/SharedDocs/FAQ/COVID-Impfen...,2022-05-24,True,NaN,NaN
842,"[Bewohner, eines, Pfelgeheims, der, Diakonie, ...",Facts For Friends,https://www.factsforfriends.de/fact/5ff85b479a...,2021-01-08,False,Falsche Behauptung,Vaccination mandate
498,"[Kinder, mit, Vorerkrankungen, sollen, weiterh...",RKI,https://www.rki.de/SharedDocs/FAQ/COVID-Impfen...,2022-06-11,True,NaN,NaN
490,"[Darüber, hinaus, sind, Abstand, halten, Maske...",RKI,https://www.rki.de/SharedDocs/FAQ/COVID-Impfen...,2022-06-03,True,NaN,NaN
...,...,...,...,...,...,...,...
110,"[Eine, Impfung, wird, allen, Personen, ab, 12,...",Zusammen gegen Corona,https://assets.ctfassets.net/eaae45wp4t29/AE5g...,2022-02-22,True,Expert panel (appointed by Government),Vaccination recommendation
293,"[Für, die, Zulassung, von, Impfstoffen, dh, di...",RKI,https://www.rki.de/SharedDocs/FAQ/COVID-Impfen...,2022-05-24,True,NaN,Security
174,"[Die, empfohlenen, Impfintervalle, betragen, b...",BMG,https://www.zusammengegencorona.de/faqs/impfen...,2022-07-10,True,Expert panel (appointed by Government),vaccination process


In [ ]:
# Remove Stop word in our Statements 
df["Statement"]= df["Statement"].apply(lambda element: [word for word in element if not word in stopWordsGerman])
df

,Statement,Source,Source (URL),Date + time,True/False,Evidence (why true?),Category
Index,,,,,,,
1,"[jugendlichen, jungen, erwachsenen, seien, kom...",ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/STIKO-emp...,2022-05-24 13-45,True,"Reliable Publisher, specialized newspaper","child vaccination, complications"
2,"[frühjahr, 2021, berichte, seltene, thrombosen...",ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/Einsatz-d...,2022-05-06 14-11,True,"Reliable Publisher, specialized newspaper",complications
3,"[immunologe, radbruch, betonte, geimpfte, gene...",ÄrzteZeitung,https://www.aerztezeitung.de/Panorama/Ueber-25...,2022-05-04 12-48,True,"Reliable Publisher, specialized newspaper",efficacy of vaccination
4,"[covid19impfstoff, spikevax, moderna, europa, ...",ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/Spikevax-...,2022-03-04 16-42,True,"Reliable Publisher, specialized newspaper","child vaccination, age limit"
5,"[zweiten, empfiehlt, stiko, impfstoff, nuvaxov...",ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/STIKO-Fue...,2022-02-03 13-34,True,"Reliable Publisher, specialized newspaper",age limit
...,...,...,...,...,...,...,...
961,"[deutsche, ärztetag, zwangsimpfungen, kindern,...",Facebook post quoted in CORRECTIV,https://correctiv.org/faktencheck/2021/05/18/c...,2021-05-18,False,Lack of source,"Vaccine mandate, kids"
962,"[pfizer, befürchtet, covid19impfstoff, geimpft...",CORRECTIV,https://correctiv.org/faktencheck/2021/05/14/c...,2021-05-14,False,"Lack of context, lack of understanding complex...","Consequences of vaccination, vaccination statu..."
963,"[aufgrund, häufung, todesfällen, gibt, ursächl...",Kla.tv quoted in CORRECTIV,https://correctiv.org/faktencheck/2021/05/12/w...,2021-05-12,False,"Lack of source, lack of understanding complex ...",Consequences of vaccination


In [ ]:
#You can join the list of above words to create a sentence without stop words, as shown below:
df["Statement"] = df["Statement"].apply(lambda x: ' '.join([word for word in x]))

df

,Statement,Source,Source (URL),Date + time,True/False,Evidence (why true?),Category
Index,,,,,,,
1,jugendlichen jungen erwachsenen seien komplika...,ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/STIKO-emp...,2022-05-24 13-45,True,"Reliable Publisher, specialized newspaper","child vaccination, complications"
2,frühjahr 2021 berichte seltene thrombosen aufr...,ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/Einsatz-d...,2022-05-06 14-11,True,"Reliable Publisher, specialized newspaper",complications
3,immunologe radbruch betonte geimpfte genesene ...,ÄrzteZeitung,https://www.aerztezeitung.de/Panorama/Ueber-25...,2022-05-04 12-48,True,"Reliable Publisher, specialized newspaper",efficacy of vaccination
4,covid19impfstoff spikevax moderna europa anwen...,ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/Spikevax-...,2022-03-04 16-42,True,"Reliable Publisher, specialized newspaper","child vaccination, age limit"
5,zweiten empfiehlt stiko impfstoff nuvaxovid gr...,ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/STIKO-Fue...,2022-02-03 13-34,True,"Reliable Publisher, specialized newspaper",age limit
...,...,...,...,...,...,...,...
961,deutsche ärztetag zwangsimpfungen kindern besc...,Facebook post quoted in CORRECTIV,https://correctiv.org/faktencheck/2021/05/18/c...,2021-05-18,False,Lack of source,"Vaccine mandate, kids"
962,pfizer befürchtet covid19impfstoff geimpften u...,CORRECTIV,https://correctiv.org/faktencheck/2021/05/14/c...,2021-05-14,False,"Lack of context, lack of understanding complex...","Consequences of vaccination, vaccination statu..."
963,aufgrund häufung todesfällen gibt ursächlichen...,Kla.tv quoted in CORRECTIV,https://correctiv.org/faktencheck/2021/05/12/w...,2021-05-12,False,"Lack of source, lack of understanding complex ...",Consequences of vaccination


In [ ]:


!pip install -U spacy                           #to install spacy-lemmatizer
!python -m spacy download de_core_news_sm     #to download the German language module

import spacy
nlp = spacy.load("de_core_news_sm")
text=open('statements3.csv').read()              #statements.csv muss auf Colab gespeichert sein

doc = nlp(text)

lemmas = []
for token in doc:
    if not token.is_punct:
        print(token, "|", token.lemma_)

In [ ]:
#to define specific lemmas yourself:
nlp.pipe_names

ar = nlp.get_pipe('attribute_ruler')

ar.add([[{"TEXT":"Myokarditis"}],[{"TEXT":"Myokarditiden"}]],{"LEMMA":"Herzmuskelentzündung"})
ar.add([[{"TEXT":"Vakzinen"}],[{"TEXT":"Vakzine"}]],{"LEMMA":"Impfung"})
ar.add([[{"TEXT":"Altersgruppe"}]],{"LEMMA":"Altersklasse"})

doc = nlp(text)
" ".join([token.lemma_ for token in doc])

In [15]:
#different way of displaying lemmas:
import pandas as pd  # für direkten Vergleich und POS-Anzeige
pd.DataFrame({"Token": [token.text for token in doc],
              "Lemma": [token.lemma_ for token in doc],
              "POS": [token.pos_ for token in doc],
              "Tag": [token.tag_ for token in doc],
              "Dep": [token.dep_ for token in doc]})

,Token,Lemma,POS,Tag,Dep
0,Index,Index,NOUN,NN,ROOT
1,",",--,PUNCT,"$,",punct
2,Statement,Statement,NOUN,NN,cj
3,",",--,PUNCT,"$,",punct
4,Source,Source,PROPN,NE,cj
...,...,...,...,...,...
41708,social,social,ADV,NE,nk
41709,/,--,PUNCT,$(,punct
41710,work,Work,PROPN,NE,cj
41711,life,lifen,ADJ,VVFIN,ROOT


#Neural Net Programming: Text Classification


In [16]:

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

In [17]:
df.shape

(999, 7)

In [18]:
df.head()

,Statement,Source,Source (URL),Date + time,True/False,Evidence (why true?),Category
Index,,,,,,,
659,"[Der, Impfstoff, kann, eine, Covid19Erkrankung...",BR24,https://www.br.de/nachrichten/wissen/fake-news...,2021-07-19 16-21,False,Lack of understanding complex medical/biologic...,Consequences of vaccination
438,"[Vorläufige, Erkenntnisse, deuten, darauf, hin...",RKI,https://www.rki.de/SharedDocs/FAQ/COVID-Impfen...,2022-05-24,True,NaN,NaN
842,"[Bewohner, eines, Pfelgeheims, der, Diakonie, ...",Facts For Friends,https://www.factsforfriends.de/fact/5ff85b479a...,2021-01-08,False,Falsche Behauptung,Vaccination mandate
498,"[Kinder, mit, Vorerkrankungen, sollen, weiterh...",RKI,https://www.rki.de/SharedDocs/FAQ/COVID-Impfen...,2022-06-11,True,NaN,NaN
490,"[Darüber, hinaus, sind, Abstand, halten, Maske...",RKI,https://www.rki.de/SharedDocs/FAQ/COVID-Impfen...,2022-06-03,True,NaN,NaN


In [19]:
print(sum(df["True/False"]==True))
print(sum(df["True/False"]==False))

502
497


In [37]:
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

# https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate/34294022
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#pattern = re.compile(r"https?://(\S+|www)\.\S+")
#for t in df["Statement"]:
#    matches = pattern.findall(t)
#    for match in matches:
#        print(t)
#        print(match)
#        print(pattern.sub(r"", t))
#    if len(matches) > 0:
#        break

#Modelling:


In [ ]:
from collections import Counter

# Count unique words
def counter_word(k):
    count = Counter()
    for text in k.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(df["Statement"])

In [ ]:
len(counter)

In [ ]:
counter

In [ ]:
counter.most_common(5)

In [ ]:
num_unique_words = len(counter)

In [ ]:
df["Target"] = df["True/False"].astype(int)

In [ ]:
df

,Statement,Source,Source (URL),Date + time,True/False,Evidence (why true?),Category,Target
Index,,,,,,,,
1,jugendlichen jungen erwachsenen seien komplika...,ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/STIKO-emp...,2022-05-24 13-45,True,"Reliable Publisher, specialized newspaper","child vaccination, complications",1
2,frühjahr 2021 berichte seltene thrombosen aufr...,ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/Einsatz-d...,2022-05-06 14-11,True,"Reliable Publisher, specialized newspaper",complications,1
3,immunologe radbruch betonte geimpfte genesene ...,ÄrzteZeitung,https://www.aerztezeitung.de/Panorama/Ueber-25...,2022-05-04 12-48,True,"Reliable Publisher, specialized newspaper",efficacy of vaccination,1
4,covid19impfstoff spikevax moderna europa anwen...,ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/Spikevax-...,2022-03-04 16-42,True,"Reliable Publisher, specialized newspaper","child vaccination, age limit",1
5,zweiten empfiehlt stiko impfstoff nuvaxovid gr...,ÄrzteZeitung,https://www.aerztezeitung.de/Medizin/STIKO-Fue...,2022-02-03 13-34,True,"Reliable Publisher, specialized newspaper",age limit,1
...,...,...,...,...,...,...,...,...
961,deutsche ärztetag zwangsimpfungen kindern besc...,Facebook post quoted in CORRECTIV,https://correctiv.org/faktencheck/2021/05/18/c...,2021-05-18,False,Lack of source,"Vaccine mandate, kids",0
962,pfizer befürchtet covid19impfstoff geimpften u...,CORRECTIV,https://correctiv.org/faktencheck/2021/05/14/c...,2021-05-14,False,"Lack of context, lack of understanding complex...","Consequences of vaccination, vaccination statu...",0
963,aufgrund häufung todesfällen gibt ursächlichen...,Kla.tv quoted in CORRECTIV,https://correctiv.org/faktencheck/2021/05/12/w...,2021-05-12,False,"Lack of source, lack of understanding complex ...",Consequences of vaccination,0


In [ ]:
# Split dataset into training and validation set
train_size = int(df.shape[0] * 0.8)
stop_size = int(df.shape[0] * 0.9)

train_df = df[:train_size]
val_df = df[train_size:]
test_df = df[stop_size:]


# split text and labels
train_sentences = train_df["Statement"].to_numpy()
train_labels = train_df["Target"].to_numpy()
val_sentences = val_df["Statement"].to_numpy()
val_labels = val_df["Target"].to_numpy()
test_sentences = test_df["Statement"].to_numpy()
test_labels = test_df["Target"].to_numpy()


In [ ]:
train_sentences.shape, val_sentences.shape , test_sentences.shape

((772,), (193,), (97,))

In [ ]:
# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training

In [ ]:
# each word has unique index
word_index = tokenizer.word_index

In [ ]:
word_index

{'impfung': 1,
 'personen': 2,
 'covid19': 3,
 'ab': 4,
 'jahren': 5,
 'impfstoff': 6,
 'auffrischimpfung': 7,
 'menschen': 8,
 'infektion': 9,
 'stiko': 10,
 'grundimmunisierung': 11,
 'impfstoffdosis': 12,
 'impfstoffe': 13,
 'comirnaty': 14,
 'geimpfte': 15,
 'risiko': 16,
 'schutz': 17,
 'covid19impfung': 18,
 'geimpften': 19,
 'sarscov2': 20,
 '2': 21,
 'daten': 22,
 'wurden': 23,
 'geimpft': 24,
 'impfungen': 25,
 'mehr': 26,
 'wurde': 27,
 'virus': 28,
 'wirksamkeit': 29,
 'empfiehlt': 30,
 'schwere': 31,
 'deutschland': 32,
 'moderna': 33,
 'schweren': 34,
 'nebenwirkungen': 35,
 'erkrankung': 36,
 'kindern': 37,
 'covid19impfstoffe': 38,
 'zeigen': 39,
 'spikevax': 40,
 'wochen': 41,
 'gegenüber': 42,
 '30': 43,
 'nuvaxovid': 44,
 'omikronvariante': 45,
 'erhalten': 46,
 'johnson': 47,
 'gibt': 48,
 'deutlich': 49,
 'laut': 50,
 'empfohlen': 51,
 'coronavirus': 52,
 'alter': 53,
 '12': 54,
 'biontechpfizer': 55,
 'mrnaimpfstoffe': 56,
 'impfstoffen': 57,
 'auftreten': 58,
 'ki

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [ ]:
print(train_sentences[10:15])
print(train_sequences[10:15])

['impfungen wären mathematischen modellierungen zufolge 314 millionen todesfälle zusammenhang coronavirus erwarten 198 millionen davon seien impfungen vermieden worden 122 millionen davon ländern hohem mittlerem einkommen'
 'impfstoff moderna menschen ab 18 jahren zugelassen'
 'coronaerkrankung verläuft kindern normalerweise eher mild dennoch cdc zufolge seit märz 2020 fünfthäufigste todesursache kindern alter vier jahren vierthäufigste kindern jahr'
 'allgemeine impfpflicht bundestag vorerst gescheitert mitarbeiter gesundheitswesen gelten strengere regeln müssen seit mitte märz impfung coronavirus vorweisen beruf tätig'
 'österreich erste land europäischen union allgemeine impfpflicht covid19 eingeführt durchgesetzt wurde nie donnerstag regierung wien impfpflicht verabschiedet']
[[25, 1116, 1117, 1118, 226, 1119, 230, 97, 271, 52, 231, 1120, 230, 158, 323, 25, 1121, 223, 1122, 230, 158, 1123, 332, 1124, 1125], [6, 33, 8, 4, 64, 5, 73], [1126, 1127, 37, 1128, 1129, 442, 1130, 443, 226,

In [ ]:
# Pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Max number of words in a sequence
max_length = 50 #longest sentence is around 46 words inclusive stop words - we can reduce it to 40 if we want to

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape , test_padded.shape

((772, 50), (193, 50), (97, 50))

In [ ]:
train_padded[10]

array([  25, 1116, 1117, 1118,  226, 1119,  230,   97,  271,   52,  231,
       1120,  230,  158,  323,   25, 1121,  223, 1122,  230,  158, 1123,
        332, 1124, 1125,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

impfungen wären mathematischen modellierungen zufolge 314 millionen todesfälle zusammenhang coronavirus erwarten 198 millionen davon seien impfungen vermieden worden 122 millionen davon ländern hohem mittlerem einkommen
[25, 1116, 1117, 1118, 226, 1119, 230, 97, 271, 52, 231, 1120, 230, 158, 323, 25, 1121, 223, 1122, 230, 158, 1123, 332, 1124, 1125]
[  25 1116 1117 1118  226 1119  230   97  271   52  231 1120  230  158
  323   25 1121  223 1122  230  158 1123  332 1124 1125    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [ ]:
# Check reversing the indices

# flip (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [ ]:
reverse_word_index # as key we have the index & the word is now the value


{1: 'impfung',
 2: 'personen',
 3: 'covid19',
 4: 'ab',
 5: 'jahren',
 6: 'impfstoff',
 7: 'auffrischimpfung',
 8: 'menschen',
 9: 'infektion',
 10: 'stiko',
 11: 'grundimmunisierung',
 12: 'impfstoffdosis',
 13: 'impfstoffe',
 14: 'comirnaty',
 15: 'geimpfte',
 16: 'risiko',
 17: 'schutz',
 18: 'covid19impfung',
 19: 'geimpften',
 20: 'sarscov2',
 21: '2',
 22: 'daten',
 23: 'wurden',
 24: 'geimpft',
 25: 'impfungen',
 26: 'mehr',
 27: 'wurde',
 28: 'virus',
 29: 'wirksamkeit',
 30: 'empfiehlt',
 31: 'schwere',
 32: 'deutschland',
 33: 'moderna',
 34: 'schweren',
 35: 'nebenwirkungen',
 36: 'erkrankung',
 37: 'kindern',
 38: 'covid19impfstoffe',
 39: 'zeigen',
 40: 'spikevax',
 41: 'wochen',
 42: 'gegenüber',
 43: '30',
 44: 'nuvaxovid',
 45: 'omikronvariante',
 46: 'erhalten',
 47: 'johnson',
 48: 'gibt',
 49: 'deutlich',
 50: 'laut',
 51: 'empfohlen',
 52: 'coronavirus',
 53: 'alter',
 54: '12',
 55: 'biontechpfizer',
 56: 'mrnaimpfstoffe',
 57: 'impfstoffen',
 58: 'auftreten',
 59:

In [ ]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [ ]:
decoded_text = decode(train_sequences[10])

print(train_sequences[10])
print(decoded_text)


[25, 1116, 1117, 1118, 226, 1119, 230, 97, 271, 52, 231, 1120, 230, 158, 323, 25, 1121, 223, 1122, 230, 158, 1123, 332, 1124, 1125]
impfungen wären mathematischen modellierungen zufolge 314 millionen todesfälle zusammenhang coronavirus erwarten 198 millionen davon seien impfungen vermieden worden 122 millionen davon ländern hohem mittlerem einkommen


#LSTM Modelling Implementation

In [ ]:
# Create LSTM model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

# Word embeddings give us a way to use an efficient, dense representation in which similar words have 
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify).

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.


model.add(layers.LSTM(64, dropout=0.1)) # dropout = .5
# potentially use auto-stopping to prevent over-fitting
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 32)            108512    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 133,409
Trainable params: 133,409
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.001) # 0.01 or 0.1
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=2)
# validation paramater is not a must, but it automatically uses the validation dataset in the training for fine-tuning.


Epoch 1/20
25/25 - 4s - loss: 0.6653 - accuracy: 0.6451 - val_loss: 0.9715 - val_accuracy: 0.0000e+00 - 4s/epoch - 158ms/step
Epoch 2/20
25/25 - 1s - loss: 0.6475 - accuracy: 0.6503 - val_loss: 1.1282 - val_accuracy: 0.0000e+00 - 767ms/epoch - 31ms/step
Epoch 3/20
25/25 - 1s - loss: 0.6508 - accuracy: 0.6503 - val_loss: 1.0423 - val_accuracy: 0.0000e+00 - 761ms/epoch - 30ms/step
Epoch 4/20
25/25 - 1s - loss: 0.6497 - accuracy: 0.6503 - val_loss: 0.9751 - val_accuracy: 0.0000e+00 - 810ms/epoch - 32ms/step
Epoch 5/20
25/25 - 1s - loss: 0.6526 - accuracy: 0.6503 - val_loss: 0.9461 - val_accuracy: 0.0000e+00 - 808ms/epoch - 32ms/step
Epoch 6/20
25/25 - 1s - loss: 0.6499 - accuracy: 0.6503 - val_loss: 1.0527 - val_accuracy: 0.0000e+00 - 751ms/epoch - 30ms/step
Epoch 7/20
25/25 - 1s - loss: 0.6471 - accuracy: 0.6503 - val_loss: 1.0137 - val_accuracy: 0.0000e+00 - 820ms/epoch - 33ms/step
Epoch 8/20
25/25 - 1s - loss: 0.5203 - accuracy: 0.7487 - val_loss: 1.7582 - val_accuracy: 0.5699 - 782ms/

In [ ]:
predictions = model.predict(train_padded)
predictions = [1 if p > 0.5 else 0 for p in predictions]


In [ ]:
print(train_sentences[10:20])

print(train_labels[10:20])
print(predictions[10:20])

In [ ]:
predictions_test = model.predict(test_padded)
predictions_test = [1 if p > 0.5 else 0 for p in predictions_test]

print(test_labels[50:100])
print(predictions_test[50:100])

# Hyperparameter - Tuning



Learning rate // Baseline model - neurons per layer and number of layers // Activation function // Regularization & dropout // Epochs


In [ ]:
pip install -U keras-tuner

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto',probability=True),
        'params' : {
            'svc__C': [1,10,100,1000],
            'svc__kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'randomforestclassifier__n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'logisticregression__C': [1,5,10]
        }
    }
}

In [ ]:
scores = []
best_estimators = {}
import pandas as pd
for algo, mp in model_params.items():
    pipe = make_pipeline(StandardScaler(), mp['model'])
    clf =  GridSearchCV(pipe, mp['params'], cv=5, return_train_score=False)
    clf.fit(train_padded, train_labels)
    scores.append({
        'model': algo,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    best_estimators[algo] = clf.best_estimator_
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.838140,"{'svc__C': 1, 'svc__kernel': 'rbf'}"
1,random_forest,0.886058,{'randomforestclassifier__n_estimators': 10}
2,logistic_regression,0.784994,{'logisticregression__C': 5}


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# We can do similar things with the GridSearch Tuner from Scikit-Learn (Tensorflow often used for DL)



In [ ]:
#Now we are building a model with an built-in hyperparameter tuner:

def build_model(hp):
  model = keras.models.Sequential()
  model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))
  #until now nothing has changed
  model.add(layers.Dense(units=hp.Int('units',min_value =32,max_value=512,step=128),activation='relu'))
  model.add(layers.LSTM(64, dropout=0.5))
  model.add(layers.Dense(1, activation="sigmoid"))
  return model
  

In [ ]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.001) # 0.01 or 0.1
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='techlabs_dl_test'
)